# Scrape latest data 

In [1]:
# import modules
from datetime import datetime
from pandas_datareader import data as pdr
import yfinance as yf
import pandas as pd
import os

In [2]:
# read stock_code.csv to get the stock code
df = pd.read_csv('stock_code.csv')
print(df.shape)

(990, 3)


In [3]:
df2 = df.to_dict('dict')
df2

{'Unnamed: 0': {0: 0,
  1: 1,
  2: 2,
  3: 3,
  4: 4,
  5: 5,
  6: 6,
  7: 7,
  8: 8,
  9: 9,
  10: 10,
  11: 11,
  12: 12,
  13: 13,
  14: 14,
  15: 15,
  16: 16,
  17: 17,
  18: 18,
  19: 19,
  20: 20,
  21: 21,
  22: 22,
  23: 23,
  24: 24,
  25: 25,
  26: 26,
  27: 27,
  28: 28,
  29: 29,
  30: 30,
  31: 31,
  32: 32,
  33: 33,
  34: 34,
  35: 35,
  36: 36,
  37: 37,
  38: 38,
  39: 39,
  40: 40,
  41: 41,
  42: 42,
  43: 43,
  44: 44,
  45: 45,
  46: 46,
  47: 47,
  48: 48,
  49: 49,
  50: 50,
  51: 51,
  52: 52,
  53: 53,
  54: 54,
  55: 55,
  56: 56,
  57: 57,
  58: 58,
  59: 59,
  60: 60,
  61: 61,
  62: 62,
  63: 63,
  64: 64,
  65: 65,
  66: 66,
  67: 67,
  68: 68,
  69: 69,
  70: 70,
  71: 71,
  72: 72,
  73: 73,
  74: 74,
  75: 75,
  76: 76,
  77: 77,
  78: 78,
  79: 79,
  80: 80,
  81: 81,
  82: 82,
  83: 83,
  84: 84,
  85: 85,
  86: 86,
  87: 87,
  88: 88,
  89: 89,
  90: 90,
  91: 91,
  92: 92,
  93: 93,
  94: 94,
  95: 95,
  96: 96,
  97: 97,
  98: 98,
  99: 99,
  100:

### Connect to database to get the latest date ( last record ) for each stock code 

In [4]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine
from urllib.parse import quote 
from datetime import timedelta

In [5]:
# connect to database
conn_string = 'postgresql://postgres:%s@localhost/dividend_investing_no2'% quote('@SMWHot4')

#connect
db = create_engine(conn_string)
conn = db.connect()
conn = psycopg2.connect(conn_string)
conn.autocommit = True
cursor = conn.cursor()

In [6]:
# using sql query to get recent date
sql2 ='''SELECT TO_CHAR (MAX("stock_date"),'YYYYMMDD') AS "latest_date","stock_code" 
FROM daily_stock
GROUP BY "stock_code"
ORDER BY "latest_date" ;'''
cursor.execute(sql2)
recent_date = cursor.fetchall()
#print(recent_date[0])
df = pd.DataFrame(recent_date, columns =['Latest Date', 'Stock Code'])
conn.commit()
conn.close()

In [7]:
df.head()

,Latest Date,Stock Code
0,20221130,5270
1,20230125,0002
2,20230125,0005
3,20230125,0006
4,20230125,0007


In [8]:
df.tail()

,Latest Date,Stock Code
985,20230125,9946
986,20230125,9954
987,20230125,9962
988,20230125,0001
989,20230125,9997


In [9]:
df['Stock Code']=df['Stock Code']+'.KL'

In [10]:
df

,Latest Date,Stock Code
0,20221130,5270.KL
1,20230125,0002.KL
2,20230125,0005.KL
3,20230125,0006.KL
4,20230125,0007.KL
...,...,...
985,20230125,9946.KL
986,20230125,9954.KL
987,20230125,9962.KL
988,20230125,0001.KL


In [11]:
# Delete the files if exist

file_path_2 = 'recent_stock.csv'

if os.path.exists(file_path_2):
    os.remove(file_path_2)
    print("recent_stock.csv removed successfully")
else:
    print("recent_stock.csv does not exist")

latest_date.csv removed successfully
recent_stock.csv removed successfully


In [12]:
for i in range(len(df)):
    
    # get each row
    lat_date = df.loc[i, "Latest Date"] 
    stock_code = df.loc[i, "Stock Code"]
    
    #20230117 or 20231223
    year = int(lat_date[0:4])    
    month = int(lat_date[4:6])
    day =  int(lat_date[6:8])
   
    
    start = datetime(year,month,day)+timedelta(days=1)
    end = datetime.now()-timedelta(days=1)
    #print(lat_date,year,month,day,stock_code, start ,end)
    try:
        if end>start:
            stock = yf.Ticker(stock_code) 
            table = stock.history(start=start, end=end)

            frame=pd.DataFrame(table)
            frame['stock code'] = stock_code
            frame.to_csv('recent_stock.csv', mode='a', index=True, header=False)
        else:
            print("Stock " + stock_code + "start and end date incorrect ")
            print("Start date :",start)
            print("End date :",end)
            print("------------------------------------------------------------")
    except:
        print("stock code: "+s+ " cannot scrape latest data")

5270.KL: No data found for this date range, symbol may be delisted


# Format and clean the data

In [13]:
# Read 'recent_stock.csv' using read_csv
find = pd.read_csv('recent_stock.csv', header = None)

In [14]:
find

,0,1,2,3,4,5,6,7,8
0,2023-01-26 00:00:00+08:00,5.950,6.050,5.950,5.950,40500,0.0,0.0,0002.KL
1,2023-01-27 00:00:00+08:00,5.950,6.130,5.950,6.130,50700,0.0,0.0,0002.KL
2,2023-01-30 00:00:00+08:00,6.130,6.130,6.020,6.020,6000,0.0,0.0,0002.KL
3,2023-01-31 00:00:00+08:00,6.030,6.030,6.000,6.010,9500,0.0,0.0,0002.KL
4,2023-02-02 00:00:00+08:00,6.020,6.200,6.020,6.200,20400,0.0,0.0,0002.KL
...,...,...,...,...,...,...,...,...,...
10853,2023-02-07 00:00:00+08:00,0.470,0.470,0.465,0.465,50000,0.0,0.0,9997.KL
10854,2023-02-08 00:00:00+08:00,0.465,0.465,0.465,0.465,6500,0.0,0.0,9997.KL
10855,2023-02-09 00:00:00+08:00,0.460,0.460,0.460,0.460,24600,0.0,0.0,9997.KL
10856,2023-02-10 00:00:00+08:00,0.460,0.460,0.460,0.460,0,0.0,0.0,9997.KL


### 1. Rename the columns header

In [15]:
# Rename the columns header
find.rename(columns = {0: 'stock_date', 1: 'stock_open', 2: 'stock_high', 3: 'stock_low', 4: 'stock_close', 5: 'stockVolume', 6: 'stockDividend', 7: 'stockSplits', 8: 'stock_code'}, inplace = True)

In [16]:
find

,stock_date,stock_open,stock_high,stock_low,stock_close,stockVolume,stockDividend,stockSplits,stock_code
0,2023-01-26 00:00:00+08:00,5.950,6.050,5.950,5.950,40500,0.0,0.0,0002.KL
1,2023-01-27 00:00:00+08:00,5.950,6.130,5.950,6.130,50700,0.0,0.0,0002.KL
2,2023-01-30 00:00:00+08:00,6.130,6.130,6.020,6.020,6000,0.0,0.0,0002.KL
3,2023-01-31 00:00:00+08:00,6.030,6.030,6.000,6.010,9500,0.0,0.0,0002.KL
4,2023-02-02 00:00:00+08:00,6.020,6.200,6.020,6.200,20400,0.0,0.0,0002.KL
...,...,...,...,...,...,...,...,...,...
10853,2023-02-07 00:00:00+08:00,0.470,0.470,0.465,0.465,50000,0.0,0.0,9997.KL
10854,2023-02-08 00:00:00+08:00,0.465,0.465,0.465,0.465,6500,0.0,0.0,9997.KL
10855,2023-02-09 00:00:00+08:00,0.460,0.460,0.460,0.460,24600,0.0,0.0,9997.KL
10856,2023-02-10 00:00:00+08:00,0.460,0.460,0.460,0.460,0,0.0,0.0,9997.KL


### 2. Remove the last three characters (.KL)

In [17]:
#Remove the last three character (.KL)
find['stock_code']=find['stock_code'].str.slice(0, -3)

In [18]:
find

,stock_date,stock_open,stock_high,stock_low,stock_close,stockVolume,stockDividend,stockSplits,stock_code
0,2023-01-26 00:00:00+08:00,5.950,6.050,5.950,5.950,40500,0.0,0.0,0002
1,2023-01-27 00:00:00+08:00,5.950,6.130,5.950,6.130,50700,0.0,0.0,0002
2,2023-01-30 00:00:00+08:00,6.130,6.130,6.020,6.020,6000,0.0,0.0,0002
3,2023-01-31 00:00:00+08:00,6.030,6.030,6.000,6.010,9500,0.0,0.0,0002
4,2023-02-02 00:00:00+08:00,6.020,6.200,6.020,6.200,20400,0.0,0.0,0002
...,...,...,...,...,...,...,...,...,...
10853,2023-02-07 00:00:00+08:00,0.470,0.470,0.465,0.465,50000,0.0,0.0,9997
10854,2023-02-08 00:00:00+08:00,0.465,0.465,0.465,0.465,6500,0.0,0.0,9997
10855,2023-02-09 00:00:00+08:00,0.460,0.460,0.460,0.460,24600,0.0,0.0,9997
10856,2023-02-10 00:00:00+08:00,0.460,0.460,0.460,0.460,0,0.0,0.0,9997


### 3. Delete unwanted columns

In [19]:
# delete unneccessary columns 
del find['stockDividend']
del find['stockSplits']
del find['stockVolume']

In [20]:
find

,stock_date,stock_open,stock_high,stock_low,stock_close,stock_code
0,2023-01-26 00:00:00+08:00,5.950,6.050,5.950,5.950,0002
1,2023-01-27 00:00:00+08:00,5.950,6.130,5.950,6.130,0002
2,2023-01-30 00:00:00+08:00,6.130,6.130,6.020,6.020,0002
3,2023-01-31 00:00:00+08:00,6.030,6.030,6.000,6.010,0002
4,2023-02-02 00:00:00+08:00,6.020,6.200,6.020,6.200,0002
...,...,...,...,...,...,...
10853,2023-02-07 00:00:00+08:00,0.470,0.470,0.465,0.465,9997
10854,2023-02-08 00:00:00+08:00,0.465,0.465,0.465,0.465,9997
10855,2023-02-09 00:00:00+08:00,0.460,0.460,0.460,0.460,9997
10856,2023-02-10 00:00:00+08:00,0.460,0.460,0.460,0.460,9997


### 4. Round to 3d.p

In [21]:
# round the columns to 3 decimal places
find['stock_open']=find['stock_open'].round(3)
find['stock_high']=find['stock_high'].round(3)
find['stock_low']=find['stock_low'].round(3)
find['stock_close']=find['stock_close'].round(3)

In [22]:
find

,stock_date,stock_open,stock_high,stock_low,stock_close,stock_code
0,2023-01-26 00:00:00+08:00,5.950,6.050,5.950,5.950,0002
1,2023-01-27 00:00:00+08:00,5.950,6.130,5.950,6.130,0002
2,2023-01-30 00:00:00+08:00,6.130,6.130,6.020,6.020,0002
3,2023-01-31 00:00:00+08:00,6.030,6.030,6.000,6.010,0002
4,2023-02-02 00:00:00+08:00,6.020,6.200,6.020,6.200,0002
...,...,...,...,...,...,...
10853,2023-02-07 00:00:00+08:00,0.470,0.470,0.465,0.465,9997
10854,2023-02-08 00:00:00+08:00,0.465,0.465,0.465,0.465,9997
10855,2023-02-09 00:00:00+08:00,0.460,0.460,0.460,0.460,9997
10856,2023-02-10 00:00:00+08:00,0.460,0.460,0.460,0.460,9997


### 5. Change data types

In [23]:
find.dtypes

stock_date      object
stock_open     float64
stock_high     float64
stock_low      float64
stock_close    float64
stock_code      object
dtype: object

In [24]:
find['stock_date'] = pd.to_datetime(find["stock_date"])
find['stock_date'] = pd.to_datetime(find["stock_date"].dt.strftime('%Y-%m-%d'))


In [25]:
find

,stock_date,stock_open,stock_high,stock_low,stock_close,stock_code
0,2023-01-26,5.950,6.050,5.950,5.950,0002
1,2023-01-27,5.950,6.130,5.950,6.130,0002
2,2023-01-30,6.130,6.130,6.020,6.020,0002
3,2023-01-31,6.030,6.030,6.000,6.010,0002
4,2023-02-02,6.020,6.200,6.020,6.200,0002
...,...,...,...,...,...,...
10853,2023-02-07,0.470,0.470,0.465,0.465,9997
10854,2023-02-08,0.465,0.465,0.465,0.465,9997
10855,2023-02-09,0.460,0.460,0.460,0.460,9997
10856,2023-02-10,0.460,0.460,0.460,0.460,9997


In [26]:
find.dtypes

stock_date     datetime64[ns]
stock_open            float64
stock_high            float64
stock_low             float64
stock_close           float64
stock_code             object
dtype: object

In [27]:
# connect to database
conn_string = 'postgresql://postgres:%s@localhost/dividend_investing_no2'% quote('@SMWHot4')

#connect
db = create_engine(conn_string)
conn = db.connect()

# use if_exists='append' to update the daily_stock table
find.to_sql("daily_stock", con=conn, if_exists='append',
          index=False,index_label=False)
conn = psycopg2.connect(conn_string)
conn.autocommit = True
cursor = conn.cursor()

conn.commit()
conn.close()

last record: 1382910
